# Black-Box Attack on a Traffic Sign Classification Model

In [43]:
import numpy as np
import warnings
from pathlib import Path
from imageio import imread
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm

from skimage.transform import resize
from skimage.color import rgb2hsv, hsv2rgb
from skimage.io import imread
from skimage.exposure import equalize_hist

from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

from keras.applications import *
from keras.models import Model

## Data Loading and Preprocessing

In [20]:
# ! mkdir data
# ! wget http://benchmark.ini.rub.de/Dataset/GTSRB_Final_Training_Images.zip -P data
# ! wget http://benchmark.ini.rub.de/Dataset/GTSRB_Final_Test_Images.zip -P data
# ! wget http://benchmark.ini.rub.de/Dataset/GTSRB_Final_Test_GT.zip -P data
# ! unzip data/GTSRB_Final_Training_Images.zip -d data
# ! unzip data/GTSRB_Final_Test_Images.zip -d data
# ! unzip data/GTSRB_Final_Test_GT.zip -d data
# ! rm data/*.zip

In [51]:
def preprocess_image(image,size=len(image)):
    # histogram normalization in v channel
    hsv = rgb2hsv(image)
    hsv[:,:,2] = equalize_hist(hsv[:,:,2])
    image = hsv2rgb(hsv)
    # central square crop
    min_side = min(image.shape[:-1])
    center = image.shape[0] // 2, image.shape[1] // 2
    image = image[center[0]-min_side//2:center[0]+min_side//2,center[1]-min_side//2:center[1]+min_side//2,:]
    # rescale to standard size
    image = resize(image, (size,size))
    # roll color axis to axis 0
    # image = np.rollaxis(image, -1)
    return image

def get_class(image_path):
    return int(str(image_path).split('/')[-2])

In [52]:
warnings.filterwarnings('ignore')

data_dir = Path('./data/GTSRB/Final_Training/Images')
images = []
labels = []

image_paths = list(data_dir.glob('*/*.ppm'))
np.random.shuffle(image_paths)
for image_path in tqdm(image_paths):
    image = imread(image_path)
    image = preprocess_image(image,size=48)
    images.append(image)
    label = get_class(image_path)
    labels.append(label)

X = np.array(images, dtype='float32')
y = np.eye(max(labels)+1, dtype='uint8')[labels]

In [53]:
# show pics of images
X.shape, y.shape

((39209, 48, 48, 3), (39209, 43))

## Training Black-Box Model

In [ ]:
# resnet-50 with finetuning

## Training Substitute Model

In [ ]:
# vgg-16 with finetuning

## Training Aversarial Examples

In [ ]:
# fast gradient method

## Attacking the Black-Box Model

In [ ]:
# get misclassification rate on adversarial samples